In [34]:
# konlpy, gensim 라이브러리 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
!python3 -m pip install --upgrade pip
!python3 -m pip install konlpy
!curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -s
!pip install gensim

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 338 kB in 1s (233 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
curl is already the newe

In [81]:
from konlpy.tag import Okt, Mecab
from gensim.models import Word2Vec

In [82]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import numpy as np

# 데이터가 큰 경우 콘솔에 모두 출력하기 위해 옵션 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [83]:
# 데이터가 큰 경우 콘솔에 모두 출력하기 위해 옵션 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 데이터 로드
games = pd.read_csv('game.csv')
tags = pd.read_csv('mechanism.csv')
game_tags = pd.read_csv('game_tag_cross.csv')
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f]

# 태그 ID와 게임 ID를 기반으로 게임 데이터와 태그 데이터 합치기
merged = pd.merge(game_tags, games, on='game_id')
merged = pd.merge(merged, tags, left_on='tag_id', right_on='tag_id')

# 한글 형태소 분석기 초기화
okt = Okt()

# 게임 별로 문서 생성
documents = {}
for game_id, game_data in merged.groupby('game_id'):
    text = game_data['game_title_eng'].iloc[0] + ' ' + ' '.join(game_data['description'])
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])

    # 한글 토큰화
    tokens = okt.morphs(text)

    # 한글 불용어 제거 (한글 불용어 리스트 필요)
    tokens = [word for word in tokens if word not in stopwords]
    documents[game_id] = tokens


In [84]:
game_vectors = {}
for game_id, tokens in documents.items():
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        game_vector = np.mean(vectors, axis=0)
        game_vectors[game_id] = game_vector

In [85]:
user_input = input("클라이언트 입력:")

클라이언트 입력:보석으로 카드를 구매하고 점수를 모으는 것을 좋아해. 세트를 모아서 시너지를 내는 것도 좋아해.


In [86]:
from gensim.models import Word2Vec

# Word2Vec 모델 학습
model = Word2Vec(sentences=list(documents.values()), vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [87]:
# 한글 불용어 제거
user_input = [word for word in user_input if word not in stopwords]


In [110]:
user_vectors = [model.wv[token] for token in user_input if token in model.wv]
if user_vectors:
    user_vector = np.mean(user_vectors, axis=0)
else:
    print("입력하신 내용을 바탕으로 추천을 할 수 없습니다.")
    exit()

In [116]:
from sklearn.metrics.pairwise import cosine_similarity

# 모든 게임에 대한 유사도 계산
similarities = {game_id: cosine_similarity([game_vector], [user_vector])[0][0] ** 2 for game_id, game_vector in game_vectors.items()}
# 유사도에 따라 게임 정렬
recommended_games = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

print(recommended_games[70])

#상위 3개 게임 추천
for game_id, similarity in recommended_games[:345]:
    game_row = games[games['game_id'] == game_id]
    if not game_row.empty:
        print(game_row['game_title_eng'].values[0], similarity)
    else:
        print("게임 ID {}를 찾을 수 없습니다.".format(game_id))


(344, 0.8259842962772304)
Scribble Time 0.8958988141276372
Pandemic 0.8907639011405344
Stockpile: Illicit Investments 0.8892682580331126
Stockpile: Continuing Corruption 0.8892457750597629
Century : A New World 0.8873688879900534
Terra Mystica 0.8872222361788751
Puerto Rico 0.8866538223178964
Century: Eastern Wonders 0.8810279268721253
Dekalko 0.8808492419811955
one card classic 0.8749630298953868
Terraforming Mars: Colonies 0.8735271769539033
Paranormal Detectives 0.8705605698857113
Raptor 0.8699859556570324
Blue Lagoon 0.8681266281646458
Ticket to Ride - Nordic Countries 0.8635717001248544
Just One 0.8631843477487564
Fake Artist Goes to New York 0.8626411817791144
Imhotep 0.8602215501231427
Ticket to Ride - Europe 0.8601174016239952
Ticket to Ride 0.8601065670042232
Gemblo 0.8598440141771881
Mr. Jack in New York 0.859745415215972
Mr. Jack 0.8595097733391732
Terraforming Mars 0.8586449616350365
Wandering Towers 0.857951504400777
Unmatched: Battle of Legends, Volume One 0.8565523012230

In [119]:
from google.colab import files
files.download('game_tag_cross.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [120]:
files.download('mechanism.csv')
files.download('game.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>